In [25]:
#import sys
#import os

#sys.path.append(os.getcwd() + '/../ffcmblens/')
#import utils, sims_generator


from orphics import lensing,io,cosmology,maps

import symlens as s
import numpy as np

from pixell import enmap

### Configuration

In [2]:
deg = 20.
px = 2.0

bin_width = 20
beam_arcmin = 1.4
noise_uk_arcmin = 7.0

#Lensing

tellmin = 30
tellmax = 3500

kellmin = 10
kellmax = 3000


### Get CMB

In [9]:
theory = cosmology.default_theory(lpad=30000)

shape,wcs = s.rect_geometry(width_deg=deg,px_res_arcmin=px)

flsims = lensing.FlatLensingSims(shape,wcs,theory,beam_arcmin,noise_uk_arcmin)

kbeam = flsims.kbeam
modlmap = enmap.modlmap(shape,wcs)
fc = maps.FourierCalc(shape,wcs)
n2d = (noise_uk_arcmin*np.pi/180./60.)**2./flsims.kbeam**2.
tmask = s.mask_kspace(shape,wcs,lmin=tellmin,lmax=tellmax)
kmask = s.mask_kspace(shape,wcs,lmin=kellmin,lmax=kellmax)
bin_edges = np.arange(kellmin,kellmax,bin_width)
binner = s.bin2D(modlmap,bin_edges)
i = 0
unlensed,kappa,lensed,beamed,noise_map,observed = flsims.get_sim(seed_cmb=(i,1),
                                                                 seed_kappa=(i,2),
                                                                 seed_noise=(i,3),
                                                                 lens_order=5,
                                                                 return_intermediate=True)
_,kmap,_ = fc.power2d(observed)
pii2d,kinput,_ = fc.power2d(kappa)

#### Observed is beamed so you have to debeam

In [4]:
kmap_deconvolved = kmap*kbeam**-1

#### Check maps

In [10]:
ells = np.arange(0,4000,1)
ucltt = theory.uCl('TT',ells)
cltt =  theory.lCl('TT',ells)